In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import nltk
from xml.etree import ElementTree
import operator

In [3]:
data_list = pd.read_csv('ace05/data/data_list.csv')
train_list = data_list[data_list.type == 'train']
test_list = data_list[data_list.type == 'test']
dev_list = data_list[data_list.type == 'dev']

In [4]:
err = []

In [5]:
train = []
for index,row in train_list.iterrows():
    with open('ace05/data/English/'+row['path']+'.apf.xml') as f:
        tree = ElementTree.parse(f)
        root = tree.getroot()

        for child in root[0]:
            if child.tag == 'event': 
                for node in child:
                    if node.tag == 'event_mention':
                        event_mention = dict()
                        event_mention['file'] = row['path']
                        event_mention['event_type'] = child.attrib['TYPE']
                        event_mention['event_subtype'] = child.attrib['SUBTYPE']
                        for child2 in node:
                            if child2.tag == 'extent':
                                charset = child2[0]
                                text_start = charset.attrib['START']
                                text_end = charset.attrib['END']
                                
                                event_mention['text'] = charset.text.replace('\n', ' ').replace('\t','')
                            if child2.tag == 'ldc_scope':
                                charset = child2[0]
                                context_start = charset.attrib['START']
                                context_end = charset.attrib['END']
                                
                                event_mention['context'] = charset.text.replace('\n', ' ').replace('\t','')
                            if child2.tag == 'anchor':
                                charset = child2[0]
                                trigger_start = charset.attrib['START']
                                trigger_end = charset.attrib['END']
                                event_mention['trigger'] = charset.text.replace('\n', ' ').replace('\t','')
                
                        event_mention['trigger_text_start'] = int(trigger_start)-int(text_start)
                        event_mention['trigger_text_end'] = int(trigger_end)-int(text_start)
                        event_mention['trigger_context_start'] = int(trigger_start)-int(context_start)
                        event_mention['trigger_context_end'] = int(trigger_end)-int(context_start)
                        
                        if '&' in event_mention['context'] and event_mention['file'] == 'wl/timex2norm/OIADVANTAGE_20050105.0922':
                            event_mention['trigger_context_start'] = 180
                            event_mention['trigger_context_end'] = 186
                        if '&' in event_mention['context']:
                            if event_mention['trigger'] != event_mention['context'][event_mention['trigger_context_start']:event_mention['trigger_context_end']+1]:
                                if event_mention['context'].count(event_mention['trigger']) > 1:
                                    err+=[event_mention]
                                else:
                                    event_mention['trigger_context_start'] = event_mention['context'].find(event_mention['trigger'])
                                    event_mention['trigger_context_end'] = event_mention['trigger_context_start'] + len(event_mention['trigger']) - 1
                            if event_mention['trigger'] != event_mention['text'][event_mention['trigger_text_start']:event_mention['trigger_text_end']+1]:
                                if event_mention['text'].count(event_mention['trigger']) > 1:
                                    err+=[event_mention]
                                else:
                                    event_mention['trigger_text_start'] = event_mention['text'].find(event_mention['trigger'])
                                    event_mention['trigger_text_end'] = event_mention['trigger_text_start'] + len(event_mention['trigger']) - 1
                        train.append(event_mention)
    

In [6]:
test = []
for index,row in test_list.iterrows():
    with open('ace05/data/English/'+row['path']+'.apf.xml') as f:
        tree = ElementTree.parse(f)
        root = tree.getroot()

        for child in root[0]:
            if child.tag == 'event': 
                for node in child:
                    if node.tag == 'event_mention':
                        event_mention = dict()
                        event_mention['file'] = row['path']
                        event_mention['event_type'] = child.attrib['TYPE']
                        event_mention['event_subtype'] = child.attrib['SUBTYPE']
                        for child2 in node:
                            if child2.tag == 'extent':
                                charset = child2[0]
                                text_start = charset.attrib['START']
                                text_end = charset.attrib['END']
                                
                                event_mention['text'] = charset.text.replace('\n', ' ').replace('\t','')
                            if child2.tag == 'ldc_scope':
                                charset = child2[0]
                                context_start = charset.attrib['START']
                                context_end = charset.attrib['END']
                                
                                event_mention['context'] = charset.text.replace('\n', ' ').replace('\t','')
                            if child2.tag == 'anchor':
                                charset = child2[0]
                                trigger_start = charset.attrib['START']
                                trigger_end = charset.attrib['END']
                                event_mention['trigger'] = charset.text.replace('\n', ' ').replace('\t','')
                                
                        event_mention['trigger_text_start'] = int(trigger_start)-int(text_start)
                        event_mention['trigger_text_end'] = int(trigger_end)-int(text_start)
                        event_mention['trigger_context_start'] = int(trigger_start)-int(context_start)
                        event_mention['trigger_context_end'] = int(trigger_end)-int(context_start)
                        
                        if '&' in event_mention['context'] and event_mention['file'] == 'wl/timex2norm/OIADVANTAGE_20050105.0922':
                            event_mention['trigger_context_start'] = 180
                            event_mention['trigger_context_end'] = 186
                            
                        if '&' in event_mention['context']:
                            if event_mention['trigger'] != event_mention['context'][event_mention['trigger_context_start']:event_mention['trigger_context_end']+1]:
                                if event_mention['context'].count(event_mention['trigger']) > 1:
                                    err+=[event_mention]
                                else:
                                    event_mention['trigger_context_start'] = event_mention['context'].find(event_mention['trigger'])
                                    event_mention['trigger_context_end'] = event_mention['trigger_context_start'] + len(event_mention['trigger']) -1
                            if event_mention['trigger'] != event_mention['text'][event_mention['trigger_text_start']:event_mention['trigger_text_end']+1]:
                                if event_mention['text'].count(event_mention['trigger']) > 1:
                                    err+=[event_mention]
                                else:
                                    event_mention['trigger_text_start'] = event_mention['text'].find(event_mention['trigger'])
                                    event_mention['trigger_text_end'] = event_mention['trigger_text_start'] + len(event_mention['trigger']) -1
                        test.append(event_mention)
    

In [7]:
dev = []
for index,row in dev_list.iterrows():
    with open('ace05/data/English/'+row['path']+'.apf.xml') as f:
        tree = ElementTree.parse(f)
        root = tree.getroot()

        for child in root[0]:
            if child.tag == 'event': 
                for node in child:
                    if node.tag == 'event_mention':
                        event_mention = dict()
                        event_mention['file'] = row['path']
                        event_mention['event_type'] = child.attrib['TYPE']
                        event_mention['event_subtype'] = child.attrib['SUBTYPE']
                        for child2 in node:
                            if child2.tag == 'extent':
                                charset = child2[0]
                                text_start = charset.attrib['START']
                                text_end = charset.attrib['END']
                                
                                event_mention['text'] = charset.text.replace('\n', ' ').replace('\t','')
                            if child2.tag == 'ldc_scope':
                                charset = child2[0]
                                context_start = charset.attrib['START']
                                context_end = charset.attrib['END']
                                
                                event_mention['context'] = charset.text.replace('\n', ' ').replace('\t','')
                            if child2.tag == 'anchor':
                                charset = child2[0]
                                trigger_start = charset.attrib['START']
                                trigger_end = charset.attrib['END']
                                event_mention['trigger'] = charset.text.replace('\n', ' ').replace('\t','')
                                
                        event_mention['trigger_text_start'] = int(trigger_start)-int(text_start)
                        event_mention['trigger_text_end'] = int(trigger_end)-int(text_start)
                        event_mention['trigger_context_start'] = int(trigger_start)-int(context_start)
                        event_mention['trigger_context_end'] = int(trigger_end)-int(context_start)
                        
                        if '&' in event_mention['context'] and event_mention['file'] == 'wl/timex2norm/OIADVANTAGE_20050105.0922':
                            event_mention['trigger_context_start'] = 180
                            event_mention['trigger_context_end'] = 186
                        if '&' in event_mention['context']:
                            if event_mention['trigger'] != event_mention['context'][event_mention['trigger_context_start']:event_mention['trigger_context_end']+1]:
                                if event_mention['context'].count(event_mention['trigger']) > 1:
                                    err+=[event_mention]
                                else:
                                    event_mention['trigger_context_start'] = event_mention['context'].find(event_mention['trigger'])
                                    event_mention['trigger_context_end'] = event_mention['trigger_context_start'] + len(event_mention['trigger']) - 1
                            if event_mention['trigger'] != event_mention['text'][event_mention['trigger_text_start']:event_mention['trigger_text_end']+1]:
                                if event_mention['text'].count(event_mention['trigger']) > 1:
                                    err+=[event_mention]
                                else:
                                    event_mention['trigger_text_start'] = event_mention['text'].find(event_mention['trigger'])
                                    event_mention['trigger_text_end'] = event_mention['trigger_text_start'] + len(event_mention['trigger']) - 1
                        dev.append(event_mention)
    

In [8]:
pd.DataFrame(train).to_csv('preprocess/train.csv',index=False)
pd.DataFrame(test).to_csv('preprocess/test.csv',index=False)
pd.DataFrame(dev).to_csv('preprocess/dev.csv',index=False)

In [435]:
train = pd.read_csv('preprocess/train.csv')
test = pd.read_csv('preprocess/test.csv')
dev = pd.read_csv('preprocess/dev.csv')

In [436]:
a = []
for index,row in train.iterrows():
    if row['trigger'] == row['text'][row['trigger_text_start']:row['trigger_text_end']+1] and row['trigger'] == row['context'][row['trigger_context_start']:row['trigger_context_end']+1]:
        continue
    else:
        a += [row]
        print("error!!!!!!!!")
        
for index,row in test.iterrows():
    if row['trigger'] == row['text'][row['trigger_text_start']:row['trigger_text_end']+1] and row['trigger'] == row['context'][row['trigger_context_start']:row['trigger_context_end']+1]:
        continue
    else:
        a += [row]
        print("error!!!!!!!!")
        
for index,row in dev.iterrows():
    if row['trigger'] == row['text'][row['trigger_text_start']:row['trigger_text_end']+1] and row['trigger'] == row['context'][row['trigger_context_start']:row['trigger_context_end']+1]:
        continue
    else:
        a += [row]
        print("error!!!!!!!!")
        